# Model Challenge

In this challenge, we will import the reduced order representations of 2000 3D microstructures generated by Voronoi tessellation, along with their corresponding property responses obtained by finite element simulations. We will then construct a Gaussian Process Regression (GPR) model to predict the property values of a testing set.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%load_ext autoreload
%autoreload 2

We need to import the data, so we will do so here:

In [ ]:
import pickle

filename='my_model_set'

def unpickle_data(filename):
    file=open(filename,'rb')
    data=pickle.load(file)
    file.close
    
    X_train=data[0]
    X_test=data[1]
    y_train=data[2]
    y_test=data[3]
    
    return X_train,X_test,y_train,y_test

X_train,X_test,y_train,y_test=unpickle_data(filename)

We have now imported the data, but we should know what we're working with. Let's take a look at the dimensions of the data:

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

As we can see, there are 200 microstructures in the training set, and 1800 microstructures in the testing set. In addition, we see that our X data has 11 dimensions, while y is unidimensional. The 11 dimensions in the X data refers to 10 PC scores plus a strength contrast (yield strength ratio of two phases). The y data is the phase-averaged equivalent strain rate of the microstructure's hard phase, usually written as $\chi_2$. This quantity is important for generating predictions of the microstructure's stress-strain curve. Next, we import the tools we need to build a GPR model.

In [ ]:
import numpy as np

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern #parameters are length_scale, length_scale_bounds, and nu (0.5,1.5,2.5)
from sklearn.gaussian_process.kernels import RBF #parameters are length_scale and length_scale_bounds
from sklearn.gaussian_process.kernels import RationalQuadratic #parameters are length_scale, alpha, length_scale_bounds, and
#alpha_bounds
from sklearn.gaussian_process.kernels import WhiteKernel #parameters are noise_level and noise_level_bounds

Now that we've imported all the tools necessary to build a GPR model, let's actually build it! Choose your kernel function wisely. There are 4 types (Matern, RBF, RationalQuadratic, WhiteKernel) of kernel functions from which to choose, and combinations are also possible. Each kernel function type has its own hyperparameters that can be adjusted by the user. For Matern and RBF functions also have the ability to specify multiple length scales. 

In [ ]:
#User inputs that define the model(s)

kernel=1.0 * Matern(length_scale=1*np.ones(11), length_scale_bounds=(1e-2, 1e6),nu=0.5) #kernel function used for GP model
local_gp=1 #select 1 to use local GP models; a global model is used for any other value
n_clusters=3 #number of clusters used over the input space if a local GP approach is used
normalize_cluster=1 #select 1 to normalize input space before defining clusters if local GP approach is used

#User inputs that do not affect the model(s) but may aid with visualization

plot_inputs=1 #select 1 to plot the training and testing sets in input space. Will show clusters if a local GP approach is used
pc_scores=[1,2] #select 2 non-identical PC scores to plot for input space visualization
plot_parity=1 #select 1 to plot the parity plot (predicted vs. actual responses)

Let's call in a function to build your model, and then let's see how well it works!

In [ ]:
from models import model_builder
model_builder(X_train,X_test,y_train,y_test,kernel,local_gp,n_clusters,normalize_cluster,plot_inputs,pc_scores,plot_parity)